In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install essentia

     |████████████████████████████████| 12.0MB 309kB/s 


In [3]:
import numpy as np
import pandas as pd
import librosa
import os
from glob import glob
from scipy.stats import norm, kurtosis, skew, entropy
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score
import essentia
import essentia.standard as es

In [4]:
class FeatureExtracter():
    df = pd.DataFrame()
    def __init__(self, path, mfcc=True, entropy=False):
        self.path = path
        self.data_files = glob(path + "/*/*.wav")
        self.spectral_features = ['spectral_centroid', 'spectral_bandwidth','spectral_contrast', 'spectral_flatness',
                    'spectral_rolloff', 'zero_crossing_rate','poly_features', 'rms']
        self.mfcc = mfcc
        self.entropy = entropy
        self.spectral_columns = []
        self.lpc_columns = []
        if mfcc:
            self.spectral_features.append('mfcc')
        
        self.genres = {v:k for k,v in enumerate([i[54:] for i in glob(path + "/*")])}
        self.fields = ['mean', 'std', 'kurtosis', 'skewness', 'median', 'min', 'max']
        if entropy:
            self.fields.append('entropy')
        
        classes = list(self.genres)

    
    def getColumnNames(self):
        features = []
        for i in self.spectral_features:
            features.append(i)
            features.append(i + "_delta1")
            features.append(i + "_delta2")

        spectral_columns = [i+"_"+j for i in features for j in self.fields]
        
        lpc_columns = ["LPC_"+i for i in self.fields]

        return spectral_columns, lpc_columns
    
    def calculate_values(self, arr):
        mean = arr.mean()
        std = arr.std()
        kurt = kurtosis(arr.T)[0]
        skewness = skew(arr.T)[0]
        median = np.median(arr)
        min_val = arr.min()
        max_val = arr.max()
        values = list((mean, std, kurt, skewness, median, min_val, max_val))
        if self.entropy:
            ent = entropy(arr)
            values.append(ent)
        return values

    def extract(self):
        spectral_df = []
        for i in tqdm(self.data_files):
            x, sr = librosa.load(i)
            line = []
            for k in self.spectral_features:
                if k == 'spectral_flatness':
                    arr = eval("librosa.feature.{}(x)".format(k))
                else:
                    arr = eval("librosa.feature.{}(x,sr)".format(k))
                arr_delta1 = librosa.feature.delta(arr, order=1)
                arr_delta2 = librosa.feature.delta(arr, order=2)
                line += self.calculate_values(arr)
                line += self.calculate_values(arr_delta1)
                line += self.calculate_values(arr_delta2)
            spectral_df.append(line.copy())

        lpc_df = []
        for i in tqdm(self.data_files):
            line = []
            audio = es.MonoLoader(filename=i)()
            lpc = es.LPC()(audio)[0]
            mean = lpc.mean()
            std = lpc.std()
            kurt = kurtosis(lpc)
            skewness = skew(lpc)
            median = np.median(lpc)
            min_val = lpc.min()
            max_val = lpc.max()
            line += [mean, std, kurt, skewness, median, min_val, max_val]
            lpc_df.append(line)
        
        spectral_columns, lpc_columns = self.getColumnNames()

        spectral_df = pd.DataFrame(data=spectral_df, columns=spectral_columns)
        lpc_df = pd.DataFrame(data=lpc_df, columns=lpc_columns)
        df = pd.concat([spectral_df, lpc_df], axis=1)
        labels = [self.genres[k] for k in [i[54:54+(i[54:].find("/"))] for i in self.data_files]]
        df['labels'] = labels.copy()
        self.df = df
        return self.df

    def save(self, path):
        self.df.to_pickle(path, protocol=4)

In [ ]:
extracter = FeatureExtracter("/content/drive/MyDrive/For_Colab/Data/genres_original",
                             mfcc=True, entropy=False)

In [ ]:
df = extracter.extract()

100%|██████████| 1000/1000 [11:30<00:00,  1.45it/s]


In [ ]:
extracter.save("/content/drive/MyDrive/For_Colab/Data/extracter.pkl")

In [5]:
df = pd.read_pickle("/content/drive/MyDrive/For_Colab/Data/extracter.pkl")
df.head()

,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_kurtosis,spectral_centroid_skewness,spectral_centroid_median,spectral_centroid_min,spectral_centroid_max,spectral_centroid_delta1_mean,spectral_centroid_delta1_std,spectral_centroid_delta1_kurtosis,spectral_centroid_delta1_skewness,spectral_centroid_delta1_median,spectral_centroid_delta1_min,spectral_centroid_delta1_max,spectral_centroid_delta2_mean,spectral_centroid_delta2_std,spectral_centroid_delta2_kurtosis,spectral_centroid_delta2_skewness,spectral_centroid_delta2_median,spectral_centroid_delta2_min,spectral_centroid_delta2_max,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_bandwidth_kurtosis,spectral_bandwidth_skewness,spectral_bandwidth_median,spectral_bandwidth_min,spectral_bandwidth_max,spectral_bandwidth_delta1_mean,spectral_bandwidth_delta1_std,spectral_bandwidth_delta1_kurtosis,spectral_bandwidth_delta1_skewness,spectral_bandwidth_delta1_median,spectral_bandwidth_delta1_min,spectral_bandwidth_delta1_max,spectral_bandwidth_delta2_mean,spectral_bandwidth_delta2_std,spectral_bandwidth_delta2_kurtosis,spectral_bandwidth_delta2_skewness,spectral_bandwidth_delta2_median,...,rms_delta1_kurtosis,rms_delta1_skewness,rms_delta1_median,rms_delta1_min,rms_delta1_max,rms_delta2_mean,rms_delta2_std,rms_delta2_kurtosis,rms_delta2_skewness,rms_delta2_median,rms_delta2_min,rms_delta2_max,mfcc_mean,mfcc_std,mfcc_kurtosis,mfcc_skewness,mfcc_median,mfcc_min,mfcc_max,mfcc_delta1_mean,mfcc_delta1_std,mfcc_delta1_kurtosis,mfcc_delta1_skewness,mfcc_delta1_median,mfcc_delta1_min,mfcc_delta1_max,mfcc_delta2_mean,mfcc_delta2_std,mfcc_delta2_kurtosis,mfcc_delta2_skewness,mfcc_delta2_median,mfcc_delta2_min,mfcc_delta2_max,LPC_mean,LPC_std,LPC_kurtosis,LPC_skewness,LPC_median,LPC_min,LPC_max
0,1459.366472,661.709451,0.841681,0.966495,1312.423565,415.633570,4499.417241,0.637114,81.736023,4.675139,0.938981,-6.190309,-259.683492,557.643613,0.024330,46.319293,4.396990,-0.544101,4.797188,-321.764147,221.536483,1389.009131,430.143278,-0.790174,0.647521,1245.152886,763.896690,2682.510435,0.209451,47.979855,1.043309,-0.135983,-0.330070,-183.963257,172.859493,0.158786,27.622576,1.208689,-0.422796,2.859325,...,2.665213,0.536220,-0.001357,-0.069095,0.070742,0.000024,0.007349,5.086851,-0.517029,0.000367,-0.051982,0.033290,-7.856247,59.767918,-0.573822,-0.192890,-6.295442,-443.449982,201.395950,0.003157,4.244223,0.753149,0.755701,-0.014800,-35.611916,61.940201,0.002818,2.340220,0.416216,-0.177388,0.008556,-29.587139,27.444580,0.000257,3.167496,0.477754,0.252205,0.119992,-5.814752,7.004394
1,1552.811865,395.559911,11.603143,1.927469,1510.972589,417.254802,4669.770713,0.341307,68.457817,8.657651,-0.177911,-2.044817,-482.515840,413.034054,0.018451,39.079206,10.091214,-0.989551,0.481424,-359.250830,182.149399,1747.702312,276.141616,0.071457,0.017533,1741.173116,960.211956,2662.739385,0.431479,49.357886,-0.236268,-0.029415,-0.261212,-151.377924,149.766938,0.092237,30.548318,-0.356420,-0.060871,0.956199,...,-0.447615,-0.071674,0.000550,-0.032720,0.031167,0.000028,0.006142,-0.013974,-0.166670,0.000172,-0.024213,0.019053,1.034164,43.077038,0.500346,-0.577687,-3.161015,-343.934814,198.838501,0.000070,3.274759,-0.185535,0.053240,0.011495,-34.185745,39.863064,-0.000213,2.004048,0.270294,0.053689,0.001813,-24.621988,24.823193,0.000194,4.738595,-0.091538,-0.206327,0.134157,-9.608500,8.883484
2,1835.004266,586.003361,1.535905,1.050190,1732.517594,849.658285,4464.156642,-0.214389,68.072488,0.329900,0.430382,-3.857220,-197.758805,235.302721,-0.282490,44.159677,0.291734,-0.134831,1.292480,-151.569984,130.421292,1748.172116,297.397392,-0.284144,0.340901,1737.639906,1081.656537,2770.695509,0.091363,34.881861,0.038399,-0.017917,-0.559164,-128.846344,97.525389,-0.072185,23.628324,0.168883,-0.128740,0.675958,...,1.094863,0.393583,-0.000821,-0.027520,0.036711,-0.000027,0.003921,1.608814,-0.446032,0.000190,-0.017889,0.012219,-8.458840,51.298744,-0.526156,-0.311698,-6.868770,-356.302917,188.284668,0.006732,3.4

In [ ]:
full_df = df.copy()

In [200]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [227]:
class ModelSelection():
    all_models = []
    def __init__(self, df, k, random_forest=True, logistic_reg=True, adaboost=True, decision_tree=True,
                 knn=True, svm=True, naive_bayes=True, mlp=True):

        self.df = df
        self.k = k
        self.random_forest = random_forest
        self.logistic_reg = logistic_reg
        self.adaboost = adaboost
        self.decision_tree = decision_tree
        self.knn = knn
        self.svm = svm
        self.naive_bayes = naive_bayes
        self.mlp = mlp

    def create_models(self):
        if self.random_forest:
            self.all_models.append(RandomForestClassifier(n_estimators=100, max_depth=10, max_features=0.6, min_samples_split=2))
            self.all_models.append(RandomForestClassifier(n_estimators=100, max_depth=20, max_features=0.6, min_samples_split=2))
            self.all_models.append(RandomForestClassifier(n_estimators=100, max_depth=10, max_features='sqrt', min_samples_split=2))
            self.all_models.append(RandomForestClassifier(n_estimators=100, max_depth=20, max_features='sqrt', min_samples_split=2))
            self.all_models.append(RandomForestClassifier(n_estimators=100, max_depth=50, max_features='auto', min_samples_split=2))
        if self.logistic_reg:
            self.all_models.append(LogisticRegression(C=10, max_iter=500, n_jobs=-1))
            self.all_models.append(LogisticRegression(C=1, max_iter=500, n_jobs=-1))
            self.all_models.append(LogisticRegression(C=0.1, max_iter=500, n_jobs=-1))
            self.all_models.append(LogisticRegression(C=0.01, max_iter=500, n_jobs=-1))
            self.all_models.append(LogisticRegression(C=0.001, max_iter=500, n_jobs=-1))
        if self.adaboost:
            self.all_models.append(AdaBoostClassifier(n_estimators=100, learning_rate=1, algorithm='SAMME.R'))
            self.all_models.append(AdaBoostClassifier(n_estimators=100, learning_rate=0.1, algorithm='SAMME.R'))
            self.all_models.append(AdaBoostClassifier(n_estimators=100, learning_rate=0.01, algorithm='SAMME.R'))
            self.all_models.append(AdaBoostClassifier(n_estimators=100, learning_rate=0.1, algorithm='SAMME'))
            self.all_models.append(AdaBoostClassifier(n_estimators=100, learning_rate=0.01, algorithm='SAMME'))
        if self.decision_tree:
            self.all_models.append(DecisionTreeClassifier(criterion='gini', max_depth=10))
            self.all_models.append(DecisionTreeClassifier(criterion='gini', max_depth=30))
            self.all_models.append(DecisionTreeClassifier(criterion='entropy', max_depth=10))
            self.all_models.append(DecisionTreeClassifier(criterion='entropy', max_depth=30))
            self.all_models.append(DecisionTreeClassifier(criterion='gini', max_depth=50))
        if self.knn:
            self.all_models.append(KNeighborsClassifier(n_neighbors=1, weights='uniform'))
            self.all_models.append(KNeighborsClassifier(n_neighbors=1, weights='distance'))
            self.all_models.append(KNeighborsClassifier(n_neighbors=2, weights='distance'))
            self.all_models.append(KNeighborsClassifier(n_neighbors=3, weights='uniform'))
            self.all_models.append(KNeighborsClassifier(n_neighbors=3, weights='distance'))
        if self.svm:
            self.all_models.append(SVC(kernel='linear'))
            self.all_models.append(SVC(kernel='poly', degree=3, C=1))
            self.all_models.append(SVC(kernel='poly', degree=4, C=1))
            self.all_models.append(SVC(kernel='poly', degree=5, C=1))
            self.all_models.append(SVC(kernel='poly', degree=5, C=0.1))
        if self.mlp:
            self.all_models.append(MLPClassifier(hidden_layer_sizes=(100,100), solver='adam', learning_rate_init=0.001, max_iter=500, early_stopping=True))
            self.all_models.append(MLPClassifier(hidden_layer_sizes=(256,256), solver='adam', learning_rate_init=0.001, max_iter=500, early_stopping=True))
            self.all_models.append(MLPClassifier(hidden_layer_sizes=(100,100), solver='lbfgs', learning_rate_init=0.001, max_iter=500, early_stopping=True))
            self.all_models.append(MLPClassifier(hidden_layer_sizes=(256,256), solver='lbfgs', learning_rate_init=0.001, max_iter=500, early_stopping=True))
            self.all_models.append(MLPClassifier(hidden_layer_sizes=(100,100), solver='lbfgs', learning_rate_init=1e-4, max_iter=500, early_stopping=True))

    def apply_models(self):
        self.all_models.clear()
        self.create_models()
        X = self.df.drop('genre',axis=1).values
        y = self.df['genre'].values
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        scores = []
        for i in tqdm(self.all_models):
            print("\n{} ------> Cross Validation Score : {}".format(i,cross_val_score(i,X,y,scoring='accuracy').mean()))
            scores.append("\n{} ------> Cross Validation Score : {}".format(i,cross_val_score(i,X,y,scoring='accuracy').mean()))
        return scores

In [228]:
from sklearn.model_selection import KFold
selector = ModelSelection(df, 5, naive_bayes=False)

In [229]:
KFold(n_splits=5)

KFold(n_splits=5, random_state=None, shuffle=False)

In [230]:
selector = ModelSelection(df, 5, naive_bayes=False)
scores = selector.apply_models()



  0%|          | 0/35 [00:00<?, ?it/s]


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=0.6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) ------> Cross Validation Score : 0.576




  3%|▎         | 1/35 [00:52<29:49, 52.64s/it]


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features=0.6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) ------> Cross Validation Score : 0.576




  6%|▌         | 2/35 [01:47<29:15, 53.18s/it]

...



 94%|█████████▍| 33/35 [05:51<00:15,  7.87s/it]


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(256, 256), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False) ------> Cross Validation Score : 0.65




 97%|█████████▋| 34/35 [06:17<00:13, 13.22s/it]


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.0001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False) ------> Cross Validation Score : 0.639




100%|██████████| 35/35 [06:27<00:00, 11.06s/it]


In [235]:
with open("/content/drive/MyDrive/For_Colab/Data/scores.txt","w") as f:
    for i in scores:
        f.write(i)

In [178]:
X_train, X_test, y_train, y_test = train_test_split(df.values, y, test_size=0.33, random_state=42)

In [179]:
from sklearn.naive_bayes import GaussianNB, CategoricalNB, MultinomialNB

In [180]:
from sklearn.preprocessing import StandardScaler

In [181]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [182]:
gnb.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100, 100), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [183]:
gnb.score(X_test, y_test)

0.5666666666666667